# Polymarket Sports Orderbook Monitor Demo

This notebook demonstrates the **polymarket_orderbook** package for monitoring and managing sports moneyline markets on Polymarket.

## Features:
- 📊 Real-time orderbook data fetching
- 🏀 Sports market filtering (NFL, NBA, UFC, etc.)
- 📈 Continuous monitoring with callbacks
- 💰 Position tracking and P&L management
- ⚠️ Stop-loss and take-profit alerts

In [1]:
# Import required libraries
import time
from polymarket_orderbook import OrderbookClient, TradeMonitor, TradeManager, SportsMarketFilter
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display

print("✅ Imports successful!")

✅ Imports successful!


## 1. Orderbook Client - Fetch Market Data

In [2]:
# Initialize the orderbook client
client = OrderbookClient(rate_limit_delay=0.5)

# Fetch all active markets
print("Fetching active markets...")
markets_df = client.get_markets(active=True, closed=False)

print(f"\n📊 Found {len(markets_df)} active markets")
print(f"\nColumns: {markets_df.columns[:10]}...")
print("\nFirst 5 markets:")
display(markets_df.select(['question', 'volume', 'liquidity', 'active']).head())

Fetching active markets...

📊 Found 20 active markets

Columns: ['id', 'question', 'conditionId', 'slug', 'resolutionSource', 'endDate', 'liquidity', 'startDate', 'image', 'icon']...

First 5 markets:


question,volume,liquidity,active
str,str,str,bool
"""Will 2025 be the hottest year …","""2131143.924455""","""45224.01744""",true
"""Will Trump deport less than 25…","""943816.364432""","""3686.66662""",true
"""Will Trump deport 250,000-500,…","""1033429.972081""","""9102.55764""",true
"""Will Trump deport 500,000-750,…","""436736.035806""","""5941.29191""",true
"""Will Trump deport 750,000-1,00…","""381719.231646""","""7538.17262""",true


## 2. Sports Market Filter - Find Moneylines

In [3]:
# Initialize sports filter
sports_filter = SportsMarketFilter()

# Filter to sports markets
sports_df = sports_filter.filter_sports_markets(markets_df)
print(f"🏀 Found {len(sports_df)} sports markets")

# Filter to moneyline markets
moneyline_df = sports_filter.filter_moneyline_markets(markets_df)
print(f"💰 Found {len(moneyline_df)} moneyline markets")

# Get sports summary
if len(sports_df) > 0:
    summary = sports_filter.get_sports_summary(markets_df)
    print("\n📈 Sports by category:")
    display(summary)

# Show some sports markets
if len(sports_df) > 0:
    print("\n🏆 Sample Sports Markets:")
    display(sports_df.select(['question', 'volume']).head(10))

🏀 Found 3 sports markets
💰 Found 0 moneyline markets

📈 Sports by category:


sport,count
str,u32
"""Other""",3



🏆 Sample Sports Markets:


question,volume
str,str
"""Will Trump & Elon reduce the d…","""175418.748584"""
"""Will Elon cut the budget by at…","""105567.266329"""
"""Will Elon cut the budget by at…","""143979.005306"""


## 3. Fetch Orderbook Data for Specific Markets

In [ ]:
# Extract token IDs from markets
sample_token_ids = []
if len(markets_df) > 0 and 'clobTokenIds' in markets_df.columns:
    for row in markets_df.head(5).iter_rows(named=True):
        tokens = row.get('clobTokenIds', [])
        if isinstance(tokens, list) and len(tokens) > 0:
            sample_token_ids.append(tokens[0])

print(f"Selected {len(sample_token_ids)} markets for orderbook analysis")

if sample_token_ids:
    # Get orderbook for first market
    token_id = sample_token_ids[0]
    print(f"\n📖 Fetching orderbook for token: {token_id}")
    
    # Get spread
    spread = client.get_spread(token_id)
    print("\n💵 Spread Analysis:")
    print(f"   Best Bid: {spread['best_bid']:.4f}")
    print(f"   Best Ask: {spread['best_ask']:.4f}")
    print(f"   Spread: {spread['spread']:.4f} ({spread['spread_pct']:.2f}%)")
    
    # Get market depth
    depth = client.get_market_depth(token_id, levels=10)
    print("\n📊 Market Depth (Top 10 levels):")
    print(f"   Bid Volume: {depth['bid_volume']:.2f}")
    print(f"   Ask Volume: {depth['ask_volume']:.2f}")
    print(f"   Total Volume: {depth['total_volume']:.2f}")
    print(f"   Imbalance: {depth['imbalance']:.2%}")
    
    # Get full orderbook snapshot
    snapshot = client.get_orderbook_snapshot(token_id)
    if len(snapshot) > 0:
        print(f"\n📸 Orderbook Snapshot ({len(snapshot)} price levels):")
        display(snapshot.head(10))
        
        # Visualize orderbook
        bids = snapshot.filter(pl.col('side') == 'BID').sort('price', descending=True).head(15)
        asks = snapshot.filter(pl.col('side') == 'ASK').sort('price').head(15)
        
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=bids['price'].to_list(),
            y=bids['size'].to_list(),
            name='Bids',
            marker_color='green'
        ))
        fig.add_trace(go.Bar(
            x=asks['price'].to_list(),
            y=asks['size'].to_list(),
            name='Asks',
            marker_color='red'
        ))
        fig.update_layout(
            title='Orderbook Depth Visualization',
            xaxis_title='Price',
            yaxis_title='Size',
            barmode='group',
            height=400
        )
        display(fig)

## 4. Compare Multiple Markets

In [ ]:
if len(sample_token_ids) >= 3:
    print(f"Comparing top {len(sample_token_ids[:5])} markets...")
    
    # Get spreads for multiple markets
    spreads_df = client.get_top_of_book(sample_token_ids[:5])
    
    if len(spreads_df) > 0:
        print("\n📊 Market Comparison:")
        display(spreads_df.select(['token_id', 'best_bid', 'best_ask', 'spread', 'spread_pct']))
        
        # Visualize spreads
        fig = px.bar(
            spreads_df,
            x='token_id',
            y='spread_pct',
            title='Bid-Ask Spread Comparison',
            labels={'spread_pct': 'Spread %', 'token_id': 'Market Token ID'},
            color='spread_pct',
            color_continuous_scale='RdYlGn_r'
        )
        fig.update_layout(height=400, xaxis_tickangle=-45)
        display(fig)

## 5. Trade Monitor - Real-time Monitoring

In [ ]:
# Initialize trade monitor
monitor = TradeMonitor(orderbook_client=client, poll_interval=5.0)

# Set up callbacks
spread_changes = []
large_orders = []

def on_spread_change(spread_data):
    """Callback when spread changes"""
    spread_changes.append(spread_data)
    print(f"📊 Spread Update: {spread_data['token_id'][:8]}... | "
          f"Bid: {spread_data['best_bid']:.4f} | "
          f"Ask: {spread_data['best_ask']:.4f} | "
          f"Spread: {spread_data['spread_pct']:.2f}%")

def on_large_order(order_data):
    """Callback when large order detected"""
    large_orders.append(order_data)
    print(f"🚨 LARGE ORDER: {order_data['side']} {order_data['size']:.0f} @ {order_data['price']:.4f}")

# Register callbacks
monitor.add_callback('spread_change', on_spread_change)
monitor.add_callback('large_order', on_large_order)

print("✅ Trade monitor configured with callbacks")

In [ ]:
# Discover sports moneyline markets to monitor
print("🔍 Discovering sports moneyline markets...")
moneyline_tokens = monitor.discover_sports_moneylines(limit=10)

if moneyline_tokens:
    print(f"✅ Found {len(moneyline_tokens)} moneyline markets")
    monitor.set_tracked_markets(moneyline_tokens[:3])  # Track first 3
else:
    # If no moneylines found, use sample tokens
    print("⚠️  No moneylines found, using sample markets")
    if sample_token_ids:
        monitor.set_tracked_markets(sample_token_ids[:3])

In [ ]:
# Start monitoring (will run for 20 seconds)
print("▶️  Starting monitor for 20 seconds...\n")
monitor.start()

# Wait and display updates
for i in range(20):
    time.sleep(1)
    if i % 5 == 0:
        print(f"\n⏱️  {20-i} seconds remaining...")

# Stop monitoring
monitor.stop()
print("\n⏸️  Monitor stopped")

In [ ]:
# Analyze collected data
spread_history_df = monitor.get_spread_dataframe()
latest_spreads_df = monitor.get_latest_spreads()

print("\n📊 Monitoring Results:")
print(f"   Total spread snapshots: {len(spread_history_df)}")
print(f"   Spread changes detected: {len(spread_changes)}")
print(f"   Large orders detected: {len(large_orders)}")

if len(latest_spreads_df) > 0:
    print("\n📈 Latest Spreads:")
    display(latest_spreads_df.select(['token_id', 'best_bid', 'best_ask', 'spread', 'spread_pct']))

# Visualize spread changes over time
if len(spread_history_df) > 0 and 'timestamp' in spread_history_df.columns:
    fig = px.line(
        spread_history_df,
        x='timestamp',
        y='spread_pct',
        color='token_id',
        title='Spread Changes Over Time',
        labels={'spread_pct': 'Spread %', 'timestamp': 'Time'}
    )
    fig.update_layout(height=400)
    display(fig)

## 6. Trade Manager - Position Tracking

In [ ]:
# Initialize trade manager
trade_manager = TradeManager(orderbook_client=client)

# Add some example positions
print("📈 Adding example positions...\n")

if sample_token_ids:
    # Add LONG position
    trade_manager.add_position(
        token_id=sample_token_ids[0],
        side="LONG",
        size=100,
        entry_price=0.55,
        market_name="Example Market 1"
    )
    
    if len(sample_token_ids) > 1:
        # Add SHORT position
        trade_manager.add_position(
            token_id=sample_token_ids[1],
            side="SHORT",
            size=50,
            entry_price=0.65,
            market_name="Example Market 2"
        )
    
    print("\n✅ Positions added")

In [ ]:
# Update positions with current prices
print("🔄 Updating positions with current market prices...\n")
trade_manager.update_positions()

# View current positions
positions_df = trade_manager.get_positions_dataframe()
print("💼 Current Positions:")
if len(positions_df) > 0:
    display(positions_df.select([
        'market_name', 'side', 'size', 'entry_price', 
        'current_price', 'unrealized_pnl', 'unrealized_pnl_pct'
    ]))

In [ ]:
# Check for alerts
print("⚠️  Checking for stop-loss and take-profit alerts...\n")
alerts = trade_manager.get_position_alerts(stop_loss_pct=-5.0, take_profit_pct=10.0)

print(f"Stop Loss Alerts: {len(alerts['stop_loss'])}")
for position in alerts['stop_loss']:
    print(f"  🔴 {position['market_name']}: {position['unrealized_pnl_pct']:.2f}%")

print(f"\nTake Profit Alerts: {len(alerts['take_profit'])}")
for position in alerts['take_profit']:
    print(f"  🟢 {position['market_name']}: {position['unrealized_pnl_pct']:.2f}%")

In [ ]:
# Simulate closing a position
if len(positions_df) > 0:
    first_token = positions_df['token_id'][0]
    exit_price = 0.60
    
    print(f"💰 Closing position at exit price: {exit_price}\n")
    pnl = trade_manager.close_position(first_token, exit_price)
    
    print(f"\n✅ Position closed with P&L: ${pnl:.2f}")

In [ ]:
# Get portfolio summary
summary = trade_manager.get_portfolio_summary()

print("\n📊 Portfolio Summary:")
print(f"   Open Positions: {summary['num_open_positions']}")
print(f"   Closed Positions: {summary['num_closed_positions']}")
print(f"   Total Unrealized P&L: ${summary['total_unrealized_pnl']:.2f}")
print(f"   Total Realized P&L: ${summary['total_realized_pnl']:.2f}")
print(f"   Total P&L: ${summary['total_pnl']:.2f}")
print(f"   Win Rate: {summary['win_rate_pct']:.1f}%")

# View P&L history
pnl_df = trade_manager.get_pnl_dataframe()
if len(pnl_df) > 0:
    print("\n📜 P&L History:")
    display(pnl_df.select([
        'market_name', 'side', 'size', 'entry_price', 
        'exit_price', 'realized_pnl', 'realized_pnl_pct'
    ]))
    
    # Visualize P&L
    fig = px.bar(
        pnl_df,
        x='market_name',
        y='realized_pnl',
        title='Realized P&L by Trade',
        labels={'realized_pnl': 'P&L ($)', 'market_name': 'Market'},
        color='realized_pnl',
        color_continuous_scale='RdYlGn'
    )
    fig.update_layout(height=400)
    display(fig)

## 7. Complete Monitoring Workflow

This cell demonstrates a complete workflow combining all components:

In [ ]:
def complete_monitoring_workflow():
    """Complete workflow for monitoring sports moneylines"""
    
    print("🚀 Starting Complete Monitoring Workflow\n")
    print("="*60)
    
    # 1. Discover markets
    print("\n1️⃣  Discovering markets...")
    client = OrderbookClient(rate_limit_delay=0.5)
    markets = client.get_markets(active=True)
    print(f"   Found {len(markets)} markets")
    
    # 2. Filter to sports moneylines
    print("\n2️⃣  Filtering to sports moneylines...")
    sports_filter = SportsMarketFilter()
    moneylines = sports_filter.filter_moneyline_markets(markets)
    print(f"   Found {len(moneylines)} moneyline markets")
    
    # 3. Set up monitoring
    print("\n3️⃣  Setting up monitoring...")
    monitor = TradeMonitor(client, poll_interval=5.0)
    
    alert_count = {"count": 0}
    
    def alert_callback(data):
        alert_count["count"] += 1
        if alert_count["count"] % 5 == 0:
            print(f"   📊 Monitoring... ({alert_count['count']} updates)")
    
    monitor.add_callback('spread_change', alert_callback)
    
    # Discover and track markets
    tokens = monitor.discover_sports_moneylines(limit=5)
    if tokens:
        monitor.set_tracked_markets(tokens[:3])
    
    # 4. Start monitoring
    print("\n4️⃣  Starting 15-second monitoring session...")
    monitor.start()
    time.sleep(15)
    monitor.stop()
    
    # 5. Analyze results
    print("\n5️⃣  Analyzing results...")
    spreads = monitor.get_latest_spreads()
    print(f"   Collected {len(spreads)} market snapshots")
    
    if len(spreads) > 0:
        print("\n📊 Final Market Status:")
        display(spreads.select(['token_id', 'best_bid', 'best_ask', 'spread_pct']).head())
    
    print("\n" + "="*60)
    print("✅ Workflow Complete!")

# Run the workflow
# complete_monitoring_workflow()  # Uncomment to run

## Summary

This notebook demonstrated the **polymarket_orderbook** package capabilities:

✅ **OrderbookClient**: Fetch real-time orderbook data, spreads, and market depth

✅ **SportsMarketFilter**: Identify and filter sports markets and moneylines

✅ **TradeMonitor**: Continuously monitor markets with callbacks for events

✅ **TradeManager**: Track positions, calculate P&L, and manage alerts

### Next Steps:
- Integrate with real trading strategies
- Add more sophisticated alert conditions
- Export data for further analysis
- Build automated trading bots